# 5-3 Self-Attension GAN の概要

## 従来の GAN の問題点
DCGAN などで用いる転置畳み込みは，局所的な情報の拡大にしかならないという問題点がある．  
可能であれば拡大する際により大域的な情報を考慮できる仕組みが望ましいと言える．

## Self-Attention の導入
上記のような問題を Self-Attention を用いて解決する．
Self-Attenition の考え方は第7章や第8章の自然言語処理でも用いられる．

ここまでで実装した G は4つの layer と last の5層のネットワークによって転置畳み込みを繰り返し画像を生成している．
この途中の layer の出力を $x$ とすると，$x$ は C×W×H（クラス数×幅×高さ）のテンソルである．
$x$ が次の layer に入力されて拡大されるとき，次式で与えられるような，より大域的な情報を考慮した $y$ を入力する．
$$
    y = x + \gamma o
$$
この式で $\gamma$ は係数，$o$ は大域的な情報を用いて $x$ を調整する変数で Self-Attention Map と呼ぶ．  
この Self-Attention Map は次のような手順で与えられる．

<img src="../image/p267.png">

はじめに，$x$ を C×W×H のテンソルを N = W×H として C×N の2次元行列に変換する．
次に $S=x^Tx$ を計算して行方向にソフトマックス関数を適用して規格化した行列を $\beta$ とする．
ここで $S$ の i 行 j 列目の要素 $S_{ij}$ は画像位置 i と j の特徴量の類似度を表す．
また，$\beta$ の各要素は次で与えられる．
$$
    \beta_{ji} = \frac{exp(S_{ij})}{\sum_{i=1}^{N}exp(S_{ij})}
$$
この $\beta$ を Attention Map と呼び，位置 j を生成する際に位置 i をどの程度考慮すべきかを表していると捉える．
$\beta$ と $x$ の積を取れば Self-Attention Map $o$ が得られる．
$$
    o_{c=i,n=j} = \sum_{k=1}^{N} x_{c=i,k} * \beta_{k,n=j}^T = \sum_{k=1}^{N} x_{c=i,k} * \beta_{n=j,k}
$$

In [2]:
import torch
import torch.nn as nn

In [3]:
# テンソルから行列への変換
X = X.view(X.shape[0], X.shape[1], X.shape[2] * X.shape[3])

# 掛け算
X_T = X.permute(0, 2, 1)
S = torch.bmm(X_T, X)

# 規格化
m = nn.Softmax(dim=-2)
attention_map_T = m(S)
attention_map = attention_map_T.permute(0, 2, 1)

# Self-Attention Map の計算
o = torch.bmm(X, attention_map.permute(0, 2, 1))

NameError: name 'X' is not defined

Self-Attention は畳み込みや転置畳み込みの計算時に着目すべきピクセルを，自分自身と似た特徴量を持つ度合いをカーネルとして毎回データごとに計算している．
自分自身の周辺にのみ着目して計算を行う通常の畳み込みや転置畳み込みでは，大域的な情報を取り込むには大きなサイズのカーネルを使う必要があり，計算量が大きくなってしまうという問題があった．
Self-Attention によって計算コストを抑えることが出来る．  
ただし，入力データ x に対してそのまま Self-Attention をかけると性能が良くなりにくいため，うまく入力データ x から大域的な情報を考慮した特徴量を計算できるように，入力データ を一度特徴量変換してから Self-Attention に与えることにする．

## 1×1 Convolution (pointwise convolution)
Self-Attention を実行する際，layer の出力 $x$ をそのまま Self-Attention の計算に使用するのではなく，一度特徴量変換してから Self-Attention に与える．
その特徴量変換の手法として，カーネルサイズが1×1の畳み込み層で $x$ をサイズ C×W×H からサイズ C'×W×H へと変換して Self-Attention で使用する．
この 1 ╳ 1 の畳み込み層のことを pointwise convolution と呼ぶ．
カーネルサイズが1なので畳み込み層から出力される結果は入力するの各チャネルを足し算するだけになる．
下図に pointwise convolution のイメージを示す．

<img src="../image/p270.png">

もし1×1の畳み込み層の出力チャネルが1つだけなら，入力をチャネルごとに線形和をとったものになる．
出力チャネルが複数であれば異なる係数でチャネルごとに線形和をとったものを，出力チャネル分だけ用意することになる．
つまり pointwise convolution は入力データのチャネルごとの線形和を作成するものであり，その1×1の畳み込み層の出力チャネル数を変化させることで元の入力 のチャネル数を C から Cʼ へと変化させる．
基本的には出力チャネル数 Cʼ は入力チャネル数 C より小さな値を取る．
このように元の入力の特徴量ごとの線形和を計算する操作は，入力 $x$ の次元圧縮をすることになる．
これによって元の入力をチャネル数 C から Cʼ へと圧縮する際に，入力の情報が欠落しにくい線形和の係数を学習している．

Self-Attention を実施する前段階でこの pointwise convolution を使用する理由は2つある．
1 つ目は Self-Attention 制限下でもうまく機能する特徴量に入力 $x$ を変換するためである．
2 つ目の理由は Self-Attention では N×C の行列 $x^T$ と C×N の行列 $x$ のかけ算をするので，その際の C を小さくして計算コストを抑えるためである．
このような Self-Attention はエッジ端末などの大きなメモリや計算能力が少ない計算機でディープラーニングを実施する MobileNets と呼ばれるモデルでも使用される．

実装で query、key、value という概念が出てくるが，これらは主に自然言語処理での SourceTarget-Attention と呼ばれる Attention 技術の概念から出てくるものである．
元の入力 x の転置に対応するものを query，元の入力 x に対応するものを key，そして Attention Map とかけ算する対象を value として，変数名を付けている．

In [4]:
# pointwise convolution を用意
query_conv = nn.Conv2d(in_channels=X.shape[1], out_channels=X.shape[1]//8, kernel_size=1)
key_conv = nn.Conv2d(in_channels=X.shape[1], out_channels=X.shape[1]//8, kernel_size=1)
value_conv = nn.Conv2d(in_channels=X.shape[1], out_channels=X.shape[1], kernel_size=1)

# 畳み込みしてからサイズを変形する（BxC'xWxH => BxC'xN）
proj_query = query_conv(X).view(X.shape[0], -1, X.shape[2] * X.shape[3])
proj_query = proj_query.permute(0, 2, 1)
proj_key = key_conv(X).view(X.shape[0], -1, X.shape[2] * X.shape[3])

# 掛け算
S = torch.bmm(proj_query, proj_key)

# 規格化
m = nn.Softmax(dim=2)
attention_map_T = m(S)
attention_map = attention_map_T.permute(0, 2, 1)

# Self-Atttention Map を計算
proj_value = value_conv(X).view(X.shape[0], -1, X.shape[2] * X.shape[3])
o = torch.bmm(proj_value, attention_map.permute(0, 2, 1))

NameError: name 'X' is not defined

## Spectral Normalization
SAGAN では Spectral Normalization を使用して，畳み込み層の重みの規格化を行う．
PyTorch の実装では torch.nn.utils.spectral_norm() となる．
Spectral Normalization はデータではなく畳み込み層などの，ネットワークの重みパラメータを規格化する操作である．  
GAN がうまく機能するには，識別器 Discriminator がリプシッツ連続性 (Lipschitz continuity) を有する必要がある．
すなわち D は「識別器 D への入力画像がほんの少し変化しても識別器 D の出力はほとんど変化しない」という特性を持つことが望ましい．

これを実現する手法が Spectral Normalization で重みを正規化するという操作になる．
Spectral Normalization は，ある層へ入力されるテンソルデータがあり，その層からの出力されるテンソルがあるとき，入力テンソルの特定の成分 (固有ベクトルに対応) が出力時に元より大きくなる場合、その特定の成分はその層の処理において拡大されることになる．  
入力画像 A と入力画像 B が存在し，画像 A と画像 B はほとんど一緒でほんの少しだけ違うとする．
このほんの少しだけ違う部分に上記の大きくなるテンソル成分を含んでいると入力画像の小さな変化は拡大されることになる
入力画像の小さな変化が拡大する状態では，これが繰り返されると識別器 D の出力手前では入力画像 A と B の小さな違いは大きな違いとなり，最終的な識別結果も大きく変わる．  
それを防ぐために層への入力テンソルのどのような成分でも出力テンソルでは拡大されることがないように，様々な成分が拡大される値のうち最大値 (最大固有値に対応) で層の重みパラメータを割り算して規格化する．
SAGAN では D だけでなく生成器 G の畳み込み層にも Spectral Normalization を使う．

In [5]:
nn.utils.spectral_norm(nn.ConvTranspose2d(z_dim, image_size * 8, kernel_size=4, stride=1))

NameError: name 'z_dim' is not defined